<a href="https://colab.research.google.com/github/OlegBEZb/NLP_advanced_course/blob/master/HW1/NLP%20advance%20course.%20HW3.%20Distributive%20semantic.%20Embedings_%2B_Bidirectional_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Embeddings to check

* Word Embeddings (word2vec, GloVe, etc.)
* Starspace and other similarity learning embeddings
* char-gram embeddings (bpemb, fasttext etc.)
* doc and sentence embeddings (doc2vec, sent2vec etc.)
* specific context models (BERT, ELMo)
* Advanced: Poincare Embeddings concept

# Libs import

In [1]:
import os, csv, random, pickle, re, sys
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import gc
from collections import Counter
import matplotlib.pyplot as plt

import gensim
from gensim.models import KeyedVectors
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

! pip install Unidecode;
from unidecode import unidecode
#! pip install pyspellchecker;
#from spellchecker import SpellChecker

from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import Activation, Dropout, LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, GRU
from tensorflow.keras.callbacks import LearningRateScheduler, Callback, ModelCheckpoint

import tensorflow as tf
%tensorflow_version 1.x
print(tf.__version__)

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
TensorFlow is already loaded. Please restart the runtime to change versions.
2.2.0-rc3


TODO: check if we can no to use this sess and use only default session

In [0]:
# # Initialize session
# sess = tf.Session()

# Check GPU

In [3]:
if tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
):
    print(tf.test.gpu_device_name())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
/device:GPU:0


# Authorization on Google drive and configurings paths

In [0]:
WORKSPACE = 'COLAB' # or 'KAGGLE'

In [5]:
if WORKSPACE == 'COLAB':
    from google.colab import drive
    drive.mount('/content/drive')

    homework_folder = os.path.join('/content/drive/My Drive', 'Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora')
    data_folder = os.path.join(homework_folder, 'Toxic data')
    embeddings_folder = os.path.join(homework_folder, 'embeddings')
    output_folder = os.path.join(homework_folder, 'output')
elif WORKSPACE == 'KAGGLE':
    data_folder = '../input/jigsaw-toxic-comment-classification-challenge/'
    embeddings_folder = '../input/glove-global-vectors-for-word-representation'
else: # TODO: add computing on premise
    pass

print('data found:', os.listdir(data_folder))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data found: ['test.csv', 'test_labels.csv', 'train.csv', 'sample_submission.csv']


In [0]:
train_filepath = os.path.join(data_folder,"train.csv")
test_filepath = os.path.join(data_folder,"test.csv")
test_labels_filepath = os.path.join(data_folder,"test_labels.csv")

#path to a sample submission
submission_path = os.path.join(data_folder,"sample_submission.csv")

# Preprocessing

## Dicts and lists of useful words and transformations

In [0]:
#List of some words that often appear in toxic comments
#Sorry about the level of toxicity in it!
toxic_words = [
    "poop", "crap", "prick", "twat", "wikipedia", "wiki", "hahahahaha", "lol",
    "bastard", "sluts", "slut", "douchebag", "douche", "blowjob", "nigga",
    "dumb", "jerk", "wanker", "wank", "penis", "motherfucker", "fucker", "fuk",
    "fucking", "fucked", "fuck", "bullshit", "shit", "stupid", "bitches",
    "bitch", "suck", "cunt", "dick", "cocks", "cock", "die", "kill", "gay",
    "jewish", "jews", "jew", "niggers", "nigger", "faggot", "fag", "asshole"
]
# todo: convert to dict and use in normalising by dict
astericks_words = [('mother****ers', 'motherfuckers'),
                   ('motherf*cking', 'motherfucking'),
                   ('mother****er', 'motherfucker'),
                   ('motherf*cker', 'motherfucker'), ('bullsh*t', 'bullshit'),
                   ('f**cking', 'fucking'), ('f*ucking', 'fucking'),
                   ('fu*cking', 'fucking'), ('****ing', 'fucking'),
                   ('a**hole', 'asshole'), ('assh*le', 'asshole'),
                   ('f******', 'fucking'), ('f*****g', 'fucking'),
                   ('f***ing', 'fucking'), ('f**king', 'fucking'),
                   ('f*cking', 'fucking'), ('fu**ing', 'fucking'),
                   ('fu*king', 'fucking'), ('fuc*ers', 'fuckers'),
                   ('f*****', 'fucking'), ('f***ed', 'fucked'),
                   ('f**ker', 'fucker'), ('f*cked', 'fucked'),
                   ('f*cker', 'fucker'), ('f*ckin', 'fucking'),
                   ('fu*ker', 'fucker'), ('fuc**n', 'fucking'),
                   ('ni**as', 'niggas'), ('b**ch', 'bitch'),
                   ('b*tch', 'bitch'), ('c*unt', 'cunt'), ('f**ks', 'fucks'),
                   ('f*ing', 'fucking'), ('ni**a', 'nigga'), ('c*ck', 'cock'),
                   ('c*nt', 'cunt'), ('cr*p', 'crap'), ('d*ck', 'dick'),
                   ('f***', 'fuck'), ('f**k', 'fuck'), ('f*ck', 'fuck'),
                   ('fc*k', 'fuck'), ('fu**', 'fuck'), ('fu*k', 'fuck'),
                   ('s***', 'shit'), ('s**t', 'shit'), ('sh**', 'shit'),
                   ('sh*t', 'shit'), ('tw*t', 'twat')]
fasttext_misspelings = {
    "'n'balls": 'balls',
    "-nazi's": 'nazis',
    'adminabuse': 'admin abuse',
    "admins's": 'admins',
    'arsewipe': 'arse wipe',
    'assfack': 'asshole',
    'assholifity': 'asshole',
    'assholivity': 'asshole',
    'asshoul': 'asshole',
    'asssholeee': 'asshole',
    'belizeans': 'mexicans',
    "blowing's": 'blowing',
    'bolivians': 'mexicans',
    'celtofascists': 'fascists',
    'censorshipmeisters': 'censor',
    'chileans': 'mexicans',
    'clerofascist': 'fascist',
    'cowcrap': 'crap',
    'crapity': 'crap',
    "d'idiots": 'idiots',
    'deminazi': 'nazi',
    'dftt': "don't feed the troll",
    'dildohs': 'dildo',
    'dramawhores': 'drama whores',
    'edophiles': 'pedophiles',
    'eurocommunist': 'communist',
    'faggotkike': 'faggot',
    'fantard': 'retard',
    'fascismnazism': 'fascism',
    'fascistisized': 'fascist',
    'favremother': 'mother',
    'fuxxxin': 'fucking',
    "g'damn": 'goddamn',
    'harassmentat': 'harassment',
    'harrasingme': 'harassing me',
    'herfuc': 'motherfucker',
    'hilterism': 'fascism',
    'hitlerians': 'nazis',
    'hitlerites': 'nazis',
    'hubrises': 'pricks',
    'idiotizing': 'idiotic',
    'inadvandals': 'vandals',
    "jackass's": 'jackass',
    'jiggabo': 'nigga',
    'jizzballs': 'jizz balls',
    'jmbass': 'dumbass',
    'lejittament': 'legitimate',
    "m'igger": 'nigger',
    "m'iggers": 'niggers',
    'motherfacking': 'motherfucker',
    'motherfuckenkiwi': 'motherfucker',
    'muthafuggas': 'niggas',
    'nazisms': 'nazis',
    'netsnipenigger': 'nigger',
    'niggercock': 'nigger',
    'niggerspic': 'nigger',
    'nignog': 'nigga',
    'niqqass': 'niggas',
    "non-nazi's": 'not a nazi',
    'panamanians': 'mexicans',
    'pedidiots': 'idiots',
    'picohitlers': 'hitler',
    'pidiots': 'idiots',
    'poopia': 'poop',
    'poopsies': 'poop',
    'presumingly': 'obviously',
    'propagandaanddisinformation': 'propaganda and disinformation',
    'propagandaministerium': 'propaganda',
    'puertoricans': 'mexicans',
    'puertorricans': 'mexicans',
    'pussiest': 'pussies',
    'pussyitis': 'pussy',
    'rayaridiculous': 'ridiculous',
    'redfascists': 'fascists',
    'retardzzzuuufff': 'retard',
    "revertin'im": 'reverting',
    'scumstreona': 'scums',
    'southamericans': 'mexicans',
    'strasserism': 'fascism',
    'stuptarded': 'retarded',
    "t'nonsense": 'nonsense',
    "threatt's": 'threat',
    'titoists': 'communists',
    'twatbags': 'douchebags',
    'youbollocks': 'you bollocks'
}
acronym_words = {
    "btw": "by the way",
    "yo": "you",
    "u": "you",
    "r": "are",
    "ur": "your",
    "ima": "i am going to",
    "imma": "i am going to",
    "i'ma": "i am going to",
    "cos": "because",
    "coz": "because",
    "stfu": "shut the fuck up",
    "wat": "what"
}
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
letter_replacement = {"$": "s", "@": "a"}
unspaced_signs = {"!": " ! ", "?": " ? "}
url_replace_dict = {
    "http://": '',
    "www.": '',
    "https://": '',
    "wikipedia.org": ''
}
#apparently, people on wikipedia love to talk about sockpuppets :-)
sockpuppets_dict = {"sock puppet": "sockpuppet", "SOCK PUPPET": "SOCKPUPPET"}

english_stopwords = set(stopwords.words('english'))

#spell_checker = SpellChecker()

cont_patterns = [
    (r'(W|w)on\'t', r'will not'),
    (r'(C|c)an\'t', r'can not'),
    (r'(I|i)\'m', r'i am'),
    (r'(A|a)in\'t', r'is not'),
    (r'(\w+)\'ll', r'\g<1> will'),
    (r'(\w+)n\'t', r'\g<1> not'),
    (r'(\w+)\'ve', r'\g<1> have'),
    (r'(\w+)\'s', r'\g<1> is'),
    (r'(\w+)\'re', r'\g<1> are'),
    (r'(\w+)\'d', r'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

#We will filter all characters except alphabet characters and some punctuation
valid_characters = " " + "@$" + "'!?-" + "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper(
)
valid_set = set(x for x in valid_characters)

## Funcs for processing

In [0]:
def replace_in_sentence(sentence, dictionary):
    for key in dictionary.keys():
        if key in sentence:
            sentence = sentence.replace(key, dictionary[key])
    return sentence


def split_toxic_and_normal_parts(word, toxic_words):
    if word == "":
        return ""

    lower = word.lower()
    for toxic_word in toxic_words:
        start = lower.find(toxic_word)
        if start >= 0:
            end = start + len(toxic_word)
            result = " ".join([
                word[0:start], word[start:end],
                split_toxic_and_normal_parts(word[end:], toxic_words)
            ])
            return result.replace("  ", " ").strip()
    return word


def normalize_by_dictionary(normalized_word, dictionary):
    """
    Word to be normalized (could be a phrase) with dictionary and the dictionary passed.
    Replaces each word in passed word-phrase into their representations in dictionary.
    Uppercased words are matched in lowercase but returned in upper. Other cases
    are checked in lower and returned in lower.
    """
    result = []
    for word in normalized_word.split():
        if word == word.upper():
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()].upper())
            else:
                result.append(word)
        else:
            #print('word', '\'', word, '\'', 'is not upper')
            #print('word.lower()', word.lower())
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()])
            else:
                result.append(word)

    return " ".join(result)


def normalize_comment(comment, tokenizer, max_chars_in_comment):
    comment = unidecode(comment)
    comment = comment[:max_chars_in_comment]

    comment = replace_in_sentence(comment, letter_replacement)
    comment = replace_in_sentence(comment, unspaced_signs)
    comment = replace_in_sentence(comment, url_replace_dict)
    comment = replace_in_sentence(comment, sockpuppets_dict)

    # ('mother****ers', 'motherfuckers')
    # for w in astericks_words:
    #     if w[0] in comment:
    #         comment = comment.replace(w[0], w[1])
    #     if w[0].upper() in comment:
    #         comment = comment.replace(w[0].upper(), w[1].upper())

    normalized_words = []
    for word in tokenizer.tokenize(comment):
        if word in english_stopwords: continue

        # # '(W|w)on\'t', r'will not'
        # for (pattern, repl) in patterns:
        #    word = re.sub(pattern, repl, word)

        if word == "." or word == ",":
            normalized_words.append(word)
            continue

        # drop url parts from links
        # word = replace_url(word)

        # replace single dots to whitespaces
        if word.count(".") == 1:
            word = word.replace(".", " ")

        # leave only legalized symbols
        filtered_word = "".join([x for x in word if x in valid_set])

        #Kind of hack: for every word check if it has a toxic word as a part of it
        #If so, split this word by swear and non-swear part.
        #normalized_word = split_toxic_and_normal_parts(filtered_word, toxic_words)
        normalized_word = filtered_word

        #         normalized_word = normalize_by_dictionary(normalized_word, hyphens_dict)
        #         normalized_word = normalize_by_dictionary(normalized_word, merged_dict)

        # check misspellings
        # temp = []
        # for word in normalized_word.split():
        #   temp.append(spell_checker.correction(word))
        # normalized_word = " ".join(temp)

        # normalized_word = normalize_by_dictionary(normalized_word, fasttext_misspelings)
        normalized_word = normalize_by_dictionary(normalized_word,
                                                  acronym_words)

        normalized_words.append(normalized_word)

    normalized_comment = " ".join(normalized_words)

    result = []
    for word in normalized_comment.split():
        # if word is upper
        if word.upper() == word:
            result.append(word)
        else:
            result.append(word.lower())

    result = " ".join(result)

    return result


def preprocess_text(df, preprocessing_func, tokenizer, max_chars_in_comment):
    # cheap and bad preprocessing
    # preprocessed_df = df[TEXT_COLUMN].fillna("CVxTz").values
    # preprocessed_df = df[TEXT_COLUMN]

    text_ndarray = df.fillna('_na').values
    np_preprocessing_func = np.vectorize(preprocessing_func)

    preprocessed_df = np_preprocessing_func(text_ndarray, tokenizer,
                                            max_chars_in_comment)

    print('Gained shape:', preprocessed_df.shape)
    return preprocessed_df


def tokenize_and_pad(text, tokenizer, max_words_in_comment):

    tokenized_text = tokenizer.texts_to_sequences(text)
    padded_text = pad_sequences(tokenized_text,
                                maxlen=max_words_in_comment,
                                dtype='int',
                                value=0)

    print('Gained shape:', padded_text.shape)
    return padded_text


def load_and_preprocess(filepath,
                        preprocessing_func,
                        tokenizer,
                        max_chars_in_comment,
                        get_labels=False):
    df = pd.read_csv(filepath)
    preprocessed_df = preprocess_text(
        df=df[TEXT_COLUMN],
        preprocessing_func=preprocessing_func,
        tokenizer=tokenizer,
        max_chars_in_comment=max_chars_in_comment)
    if get_labels:
        labels = df[TARGET_COLS].values
        return preprocessed_df, labels
    else:
        return preprocessed_df


def preproc2tokenized(preprocessed_df,
                      max_tokens,
                      max_words_in_comment,
                      tokenizer=None):
    # keras text tokenizer. simple, only for splitting and padding
    if not tokenizer:
        # from tensorflow.keras.preprocessing.text import Tokenizer
        tokenizer = Tokenizer(num_words=MAX_TOKENS) 
        tokenizer.fit_on_texts(list(preprocessed_df))
    tokenized_df = tokenize_and_pad(preprocessed_df, tokenizer,
                                    max_words_in_comment)
    return tokenized_df, tokenizer


def get_preprocessed_dfs(train_filepath,
                         test_filepath,
                         preprocessing_func,
                         max_chars_in_comment,
                         max_tokens,
                         max_words_in_comment,
                         return_tokenized=False):

    preprocessing_tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)
    preprocessed_train, train_labels = load_and_preprocess(
        train_filepath,
        preprocessing_func,
        preprocessing_tknzr,
        max_chars_in_comment,
        get_labels=True)
    preprocessed_test = load_and_preprocess(test_filepath, preprocessing_func,
                                            preprocessing_tknzr,
                                            max_chars_in_comment)

    if return_tokenized:
        # fit tokenizer
        tokenized_train, tokenizer = preproc2tokenized(
            preprocessed_train,
            max_tokens=max_tokens,
            max_words_in_comment=max_words_in_comment,
            tokenizer=None)
        # use fitted tokenizer
        tokenized_test, _ = preproc2tokenized(
            preprocessed_test,
            max_tokens=max_tokens,
            max_words_in_comment=max_words_in_comment,
            tokenizer=tokenizer)
        return tokenized_train, tokenized_test, train_labels, tokenizer
    else:
        return preprocessed_train, preprocessed_test, train_labels

## Loading frames and processing them

In [0]:
TEXT_COLUMN = 'comment_text'
TARGET_COLS = [
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
]

MAX_WORDS_IN_COMMENT = 220
MAX_TOKENS = 20000
MAX_CHARS_IN_COMMENT = 20000  #We are going to truncate a comment if its length > threshold

In [14]:
%%time

preprocessed_train, preprocessed_test, train_labels, tokenizer = get_preprocessed_dfs(
    train_filepath,
    test_filepath,
    normalize_comment,
    max_chars_in_comment=MAX_CHARS_IN_COMMENT,
    max_tokens=MAX_TOKENS,
    max_words_in_comment=MAX_WORDS_IN_COMMENT,
    return_tokenized=True)
gc.collect()

print(preprocessed_train[:3])

Gained shape: (159571,)
Gained shape: (153164,)
Gained shape: (159571, 220)
Gained shape: (153164, 220)
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0 

# Callbacks for keras models

TODO: found how to evaluate score outside the callbacks and not to evaluate twice

In [0]:
# class RocAucEvaluation(Callback):
#     def __init__(self, validation_data=(), interval=1):
#         super(Callback, self).__init__()

#         self.interval = interval
#         self.X_val, self.y_val = validation_data

#     def on_epoch_end(self, epoch, logs={}):
#         if epoch % self.interval == 0:
#             y_pred = self.model.predict(self.X_val, verbose=0)
#             score = 0
#             for i in range(6):
#              score += roc_auc_score(self.y_val[:,i], y_pred[:,i])/6.
#             print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

# RocAuc_val = RocAucEvaluation(validation_data=(x_valid, y_valid), interval = 1)


class ModelRocAucCheckpoint(Callback):
    def __init__(self, filepath, validation_data, verbose=0, digits=3,
                 save_best_only=False, save_weights_only=False,
                 save_freq=1):
        self.X_val, self.y_val = validation_data
        self.verbose = verbose
        self.digits = digits
        self.filepath = filepath
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.save_freq = save_freq
        self.epochs_since_last_save = 0

        self.monitor_op = np.greater
        self.best = -np.Inf


    def RocAucEvaluation(self):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = 0
        for i in range(6):
            score += roc_auc_score(self.y_val[:, i], y_pred[:, i]) / 6.
        score = round(score, self.digits)
        return score


    def on_epoch_end(self, epoch, logs=None):
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.save_freq:
            self.epochs_since_last_save = 0
            filepath = self.filepath
            if self.save_best_only:
                current = self.RocAucEvaluation()
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('\nEpoch %d: %s improved from %0.5f to %0.5f,'
                              ' saving model to %s'
                              % (epoch + 1, 'Roc Auc score', self.best,
                                  current, filepath))
                    self.best = current
                    if self.save_weights_only:
                        self.model.save_weights(filepath, overwrite=True)
                    else:
                        self.model.save(filepath, overwrite=True)
                else:
                    if self.verbose > 0:
                        print('\nEpoch %d: %s did not improve from %0.5f' %
                              (epoch + 1, 'Roc Auc score', self.best))
            else:
                if self.verbose > 0:
                    print('\nEpoch %d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                else:
                    self.model.save(filepath, overwrite=True)


class RocAucEarlyStopping(Callback):
    """Callback for early stopping based on roc auc on the validation set"""
    # TODO: add restore_best_weights param
    def __init__(self, validation_data, patience=0, digits=3):
        super().__init__()
        self.X_val, self.y_val = validation_data
        self.best = 0
        self.patience = patience
        self.digits = digits
        self.current_patience = patience
        

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = 0
        for i in range(6):
            score += roc_auc_score(self.y_val[:, i], y_pred[:, i]) / 6.

        score = round(score, self.digits)
        print("\nROC-AUC - epoch: {:d} - score: {}".format(epoch + 1, score))

        # check digits
        if np.greater(score, self.best):
            self.best = score
            self.current_patience = self.patience
        else:
            print('\nbest:{}\ncurrent:{}'.format(self.best, score))
            self.current_patience -= 1
            if self.current_patience < 0:
                self.model.stop_training = True
                print('Early stopping due to lower roc auc')
                self.current_patience = self.patience
            else:
                print('{} patience remained'.format(self.current_patience))

# Preparing features and training model

## Embedding matrix and Bidirectional_GRU model

### Preparing features

In [15]:
embedding_len = 1536

word_index = tokenizer.word_index
print('Found %s unique tokens.' %len(word_index))

#del tokenizer
gc.collect()

num_words = min(MAX_TOKENS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_len)) # zeroth row for UNK
print('embedding matrix shape', embedding_matrix.shape)

Found 184281 unique tokens.
embedding matrix shape (20000, 1536)


#### GloVe

In [0]:
# embedding_len = 200
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.twitter.27B.{}d.txt".format(embedding_len)), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

# embedding_len = 300
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.840B.300d.txt"), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)


# vocab_words = words[words.index.isin(list(word_index.keys()))]
# print('vocab_words.shape', vocab_words.shape)

# del words
# gc.collect()

In [0]:
# def get_vec(word, words_df):
#     return words_df.loc[word].values



# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word, vocab_words)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector

# # %%time
# # embedding_matrix = np.load('../input/embedding-2/embedding_matrix_big.npy')
    
# print('embedding_matrix.shape', embedding_matrix.shape)

#### StarSpace

In [0]:
# Starspace attempt

# from sklearn.feature_extraction.text import CountVectorizer
# count_vectorizer = CountVectorizer(min_df=0, max_df=0.99, max_features=10000)
# X_train = count_vectorizer.fit_transform(article_contents.main_content.iloc[0:train_row])
# X_train = count_vectorizer.inverse_transform(X_train)
# label_prefix = '__label__'
# with open("train_starspace.txt", 'w+') as file:
#     for i in range(10):
#         file.write(' '.join(preprocessed_train[i].split(' ')) + ' ' + label_prefix + train_labels[i])
#         file.write('\n')
# file.close()

# The result file will look like this (all separeted by space, and label will have prefix __label__)
# how are you ... __label__b
# this is just an example ... __label__c

#### FastText

In [0]:
!pip install fasttext

In [0]:
# import codecs

# #load embeddings
# print('loading word embeddings...')
# embeddings_index = {}
# f = codecs.open(os.path.join(embeddings_folder, 'crawl-300d-2M-subword.vec'), encoding='utf-8')
# for line in tqdm(f, mininterval=5):
#     values = line.rstrip().rsplit(' ')
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('found %s word vectors' % len(embeddings_index))
# print('preparing embedding matrix...')
# words_not_found = []


# for word, i in word_index.items():
#     if i >= max_tokens:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if (embedding_vector is not None) and len(embedding_vector) > 0:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#     else:
#         words_not_found.append(word)
# print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


# words_not_found[:10]

#### BPEmb

In [0]:
# !pip install bpemb;
# from bpemb import BPEmb
# # load English BPEmb model with default vocabulary size of max_tokens
# bpemb_en = BPEmb(lang="en", dim=embedding_len, vs=max_tokens)
# bpemb_en.vectors.shape

In [0]:
# # mean vector of word parts
# def get_vec(word):
#     vec = bpemb_en.embed(word)
#     return vec.mean(axis=0)

In [0]:
# %%time

# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector

# print('embedding_matrix.shape', embedding_matrix.shape)

embedding_matrix.shape (20000, 300)
CPU times: user 504 ms, sys: 18.1 ms, total: 522 ms
Wall time: 520 ms


#### Flair

In [16]:
! pip install tiny-tokenizer
! pip install flair

import flair
from flair.data import Sentence
import torch

  Created wheel for tiny-tokenizer: filename=tiny_tokenizer-3.1.0-cp36-none-any.whl size=10549 sha256=6fa3b5c9c1947dc7732086329eea5352f9831d035aaf6d308d8277e282c11ef7
  Stored in directory: /root/.cache/pip/wheels/d1/c8/36/334497a689fab90128232e86b5829b800dd271a3d5d5959c53
Successfully built tiny-tokenizer
     |████████████████████████████████| 143kB 3.4MB/s 
     |████████████████████████████████| 256kB 48.6MB/s 
     |████████████████████████████████| 573kB 50.2MB/s 
     |████████████████████████████████| 798kB 33.2MB/s 
     |████████████████████████████████| 983kB 30.8MB/s 
     |████████████████████████████████| 3.7MB 30.9MB/s 
     |████████████████████████████████| 1.0MB 28.5MB/s 
     |████████████████████████████████| 890kB 18.6MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=5bec94807b83662787e9c79677cb94b3294137a02059651655d1dc99e3847c62
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3c

##### ELMo

In [0]:
! pip install allennlp

In [0]:
from flair.embeddings import ELMoEmbeddings

# init embedding
embedding = ELMoEmbeddings('medium')

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
embedding.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)


def get_comment_emb(comment):
  sentence = Sentence(comment)
  embedding.embed(sentence)
  emb_list = []
  for word in sentence:
    emb_list.append(word.embedding)
  return emb_list

get_comment_emb_vec = np.vectorize(get_comment_emb)

embedding len 1536


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= max_tokens:
        continue
    try:
        word_sent = Sentence(word)
        embedding.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

100%|██████████| 184924/184924 [03:02<00:00, 1014.44it/s]


In [0]:
del embedding
gc.collect()

648

In [0]:
# # creating a tensor for storing sentence embeddings #
# s = torch.zeros(0, embedding_len)

# # iterating Sentence (tqdm tracks progress) #
# for tweet in tqdm(preprocessed_train[10]):   
#   # empty tensor for words #
#   w = torch.zeros(0, embedding_len)   
#   sentence = Sentence(tweet)
#   embedding.embed(sentence)
#   # for every word #
#   for token in sentence:
#     # storing word Embeddings of each word in a sentence #
#     w = torch.cat((w,token.embedding.view(-1, embedding_len)),0)
#   # storing sentence Embeddings (mean of embeddings of all words)   #
#   s = torch.cat((s, w.mean(dim = 0).view(-1, embedding_len)),0)

##### RoBERTa

In [0]:
from flair.embeddings import RoBERTaEmbeddings

In [0]:
# init embedding
embedding = RoBERTaEmbeddings('roberta-base')

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
embedding.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)

embedding len 768


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= max_tokens:
        continue
    try:
        word_sent = Sentence(word)
        embedding.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del embedding
gc.collect()

100%|██████████| 184924/184924 [04:29<00:00, 685.45it/s]


648

##### RoBERTa + ELMo

In [17]:
! pip install allennlp
from flair.embeddings import StackedEmbeddings, RoBERTaEmbeddings, ELMoEmbeddings, BertEmbeddings

     |████████████████████████████████| 7.6MB 3.2MB/s 
     |████████████████████████████████| 133kB 44.4MB/s 
     |████████████████████████████████| 256kB 42.9MB/s 
     |████████████████████████████████| 163kB 38.8MB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 30.9MB 100kB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 5.2MB 43.0MB/s 
     |████████████████████████████████| 2.1MB 38.9MB/s 
     |████████████████████████████████| 3.2MB 36.1MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.15.0-cp36-cp36m-linux_x86_64.whl size=3319808 sha256=83d3bfbbf9a8c6627180b0a3f985788cc65f4938868fd8bd73ebd1cf7d25abb9
  Stored in directory: /root/.cache/pip/wheels/57/63/2e/da89cfe1ba08550bd7262d5d9c027edc313980c3b85b3b0a38
  Creat

In [18]:
elmo_embeddings = ELMoEmbeddings('small')
roberta_embeddings = RoBERTaEmbeddings('roberta-base')
stacked_embeddings = StackedEmbeddings([
                                        elmo_embeddings,
                                        roberta_embeddings,
                                       ])

# create a sentence #
sentence = Sentence('test sent')
# embed words in sentence #
stacked_embeddings.embed(sentence)
embedding_len = sentence[0].embedding.size()[0]
print('embedding len', embedding_len)

100%|██████████| 54402456/54402456 [00:01<00:00, 33076963.27B/s]



embedding len 1536


In [0]:
for word, i in tqdm(word_index.items(), total=len(word_index)):
    if i >= MAX_TOKENS:
        continue
    try:
        word_sent = Sentence(word)
        stacked_embeddings.embed(word_sent)
        embedding_vector = word_sent[0].embedding.cpu().detach().numpy()
        if (embedding_vector is not None) and len(embedding_vector) > 0:
           embedding_matrix[i] = embedding_vector
    except KeyError:
        pass
        #embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

100%|██████████| 184924/184924 [11:48<00:00, 260.89it/s]


In [0]:
del stacked_embeddings
gc.collect()

549

### Build and fit the Bidirectional_GRU model

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(tokenized_train,
                                                      train_labels,
                                                      test_size=0.1,
                                                      shuffle=True)

print('x_train shape:', x_train.shape)
print('x_valid shape:', x_valid.shape)
print('y_train shape:', y_train.shape)
print('y_valid shape:', y_valid.shape)

del tokenized_train
gc.collect()

x_train shape: (143613, 220)
x_valid shape: (15958, 220)
y_train shape: (143613, 6)
y_valid shape: (15958, 6)


0

In [0]:
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
GLOBAL_EPOCHS = 2
EPOCHS = 5

In [0]:
def build_model(embedding_matrix):
    words = Input(shape=(MAX_WORDS_IN_COMMENT,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    #x = Input(batch_shape=(batch_size, MAX_CHARS_IN_COMMENT, embedding_len))
    
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(len(TARGET_COLS), activation='sigmoid')(hidden)
    
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [0]:
%%time

SEEDS = 2
pred = 0

for ii in tqdm(range(SEEDS), total=SEEDS):

    model = build_model(embedding_matrix)
    for global_epoch in range(GLOBAL_EPOCHS):
        print('\nglobal_epoch', global_epoch)

        model.fit(
            x_train,
            y_train,
            validation_data=(x_valid, y_valid),
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1,
            callbacks=[
                LearningRateScheduler(lambda _: 1e-3 * (0.5**global_epoch)),
                #RocAuc_val
                RocAucEarlyStopping(validation_data=(x_valid, y_valid),
                                    patience=1,
                                    digits=4)
            ])
        print('fitted')

    pred += model.predict(tokenized_test, batch_size=1024, verbose=1) / SEEDS
    np.save('pred', pred)
    model.save_weights('model_weights_' + str(ii) + '.h5')
    os.system('gzip ' + 'model_weights_' + str(ii) + '.h5')

  0%|          | 0/2 [00:00<?, ?it/s]











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

global_epoch 0


Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 87s 605us/step - loss: 0.0700 - val_loss: 0.0497
ROC-AUC - epoch: 1 - score: 0.9787

Epoch 2/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0473 - val_loss: 0.0446
ROC-AUC - epoch: 2 - score: 0.9856

Epoch 3/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0429 - val_loss: 0.0445
ROC-AUC - epoch: 3 - score: 0.9868

Epoch 4/5
143613/143613 [==============================] - 82s 568us/step - loss: 0.0402 - val_loss: 0.0428
ROC-AUC - epoch: 4 - score: 0.9879

Epoch 5/5
143613/143613 [==============================] - 82s 569us/step - loss: 0.0382 - val_loss: 0.0447
ROC-AUC - epoch: 5 - score

 50%|█████     | 1/2 [11:51<11:51, 711.77s/it]


global_epoch 0
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 82s 570us/step - loss: 0.0693 - val_loss: 0.0518
ROC-AUC - epoch: 1 - score: 0.98


best:0.9881
current:0.98
0 patience remained
Epoch 2/5
143613/143613 [==============================] - 81s 567us/step - loss: 0.0469 - val_loss: 0.0469
ROC-AUC - epoch: 2 - score: 0.9854


best:0.9881
current:0.9854
Early stopping due to lower roc auc
fitted

global_epoch 1
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 81s 567us/step - loss: 0.0417 - val_loss: 0.0426
ROC-AUC - epoch: 1 - score: 0.9873


best:0.9881
current:0.9873
0 patience remained
Epoch 2/5
143613/143613 [==============================] - 81s 566us/step - loss: 0.0395 - val_loss: 0.0427
ROC-AUC - epoch: 2 - score: 0.9878


best:0.9881
current:0.9878
Early stopping due to lower roc auc
fitted
153164/153164 [==============================] - 33s 213

100%|██████████| 2/2 [18:48<00:00, 623.22s/it]

CPU times: user 10min 36s, sys: 4min, total: 14min 36s
Wall time: 18min 48s


## BERT fine-tuned

In [0]:
import tensorflow_hub as hub
print(hub.__version__)

#Installing BERT module
!pip install bert-tensorflow

#Importing BERT modules
import bert
from bert import run_classifier, optimization, tokenization

0.7.0
     |████████████████████████████████| 71kB 2.1MB/s 



In [0]:
def create_examples(df, labels_available=True):
    """
    Creates input examples for the sets of texts and labels.
    https://github.com/google-research/bert/blob/cc7051dc592802f501e8a6f71f8fb3cf9de95dc9/run_classifier.py#L127    
    """
    examples = []
    for (i, row) in enumerate(df.values):
        guid = None  #row[0]
        text_a = row[0]
        if labels_available:
            labels = row[1:].tolist()
        else:  # what should be here in test phase
            labels = [0, 0, 0, 0, 0, 0]
        examples.append(
            bert.run_classifier.InputExample(guid=guid,
                                             text_a=text_a,
                                             label=labels))
    return examples

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"


def create_tokenizer_from_hub_module(bert_path):
    """Get the vocab file and casing info from the Hub module."""
    bert_module = hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info",
                                    as_dict=True)
    vocab_file, do_lower_case = sess.run([
        tokenization_info["vocab_file"],
        tokenization_info["do_lower_case"],
    ])
    print('loading tokenizer with{} lowercasing'.format('out' *
                                                        (not do_lower_case)))

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file,
                                           do_lower_case=do_lower_case)


bert_tokenizer = create_tokenizer_from_hub_module(BERT_MODEL_HUB)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


loading tokenizer without lowercasing



In [0]:
#todo: split to funct like here https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22
class InputFeatures(object):
    """
    A single set of features of data.
    https://github.com/google-research/bert/blob/cc7051dc592802f501e8a6f71f8fb3cf9de95dc9/run_classifier.py#L161
    The only difference from the original implementation (line above) is label_ids instead of label_id because
    here we solve multi-label not multi-class task.
    """
    def __init__(self,
                 input_ids,
                 input_mask,
                 segment_ids,
                 label_ids,
                 is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example = is_real_example


# split like here
# https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22
def convert_examples_to_features(examples, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in tqdm(enumerate(examples), total=len(examples)):

        #print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)
        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        """
        # in original repo
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)
        """
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)
        """
        # in original repo
        if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)
        """
        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        '''
        # in original repo
        while len(input_ids) < max_seq_length:
          input_ids.append(0)
          input_mask.append(0)
          segment_ids.append(0)
        '''
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        labels_ids = []
        for label in example.label:
            labels_ids.append(int(label))

        if ex_index < 2:  # make as param with default value
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x)
                                                    for x in input_ids]))
            logger.info("input_mask: %s" %
                        " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s" %
                        " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.label, labels_ids))

        features.append(
            InputFeatures(input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_ids=labels_ids))
    return features

In [0]:
def features_to_arrays(features):
    """Convert a list of InputFeatures to np.arrays"""

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)

    return (np.array(all_input_ids, dtype='int32'), 
            np.array(all_input_mask, dtype='int32'), 
            np.array(all_segment_ids, dtype='int32'))

In [0]:
def text2input_arrays(text, tokenizer, max_seq_length, labels_available, del_text=True):
    examples = create_examples(text, labels_available=labels_available)
    features = convert_examples_to_features(examples,  max_seq_length, tokenizer)
    input_ids, input_masks, segment_ids = features_to_arrays(features)

    if del_text:
        del text
        gc.collect()

    return input_ids, input_masks, segment_ids

In [0]:
preprocessed_train_with_labels = pd.concat([
    pd.DataFrame(preprocessed_train, columns=['preprocessed_text']),
    pd.DataFrame(train_labels, columns=TARGET_COLS)
], axis=1)
preprocessed_train_with_labels.head(2)

,preprocessed_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation why edits made username hardcore m...,0,0,0,0,0,0
1,d'aww ! he matches background colour i'm seemi...,0,0,0,0,0,0


In [0]:
del preprocessed_train
gc.collect()

In [0]:
%%time

train_input_ids, train_input_masks, train_segment_ids = text2input_arrays(
    text=preprocessed_train_with_labels,
    tokenizer=bert_tokenizer,
    max_seq_length=MAX_WORDS_IN_COMMENT,
    labels_available=True)


CPU times: user 2min 19s, sys: 1.57 s, total: 2min 20s
Wall time: 2min 20s


In [0]:
train_input_ids, valid_input_ids, \
train_input_masks, valid_input_masks, \
train_segment_ids, valid_segment_ids, \
train_labels, valid_labels = \
train_test_split(train_input_ids, train_input_masks, train_segment_ids, train_labels, test_size = 0.1, shuffle=True)

TODO: create pipeline which uses features as is or without data->features->arrays but data->arrays

In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(self,
                 bert_path,
                 verbose=True,
                 n_fine_tune_layers=10,
                 **kwargs):
        self.bert_path = bert_path
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        # parse it with regular expression
        self.output_size = 768
        self.verbose = verbose
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(self.bert_path,
                               trainable=self.trainable,
                               name="{}_module".format(self.name))
        trainable_vars = self.bert.variables

        # Remove unused layers
        trainable_vars = [
            var for var in trainable_vars if not "/cls/" in var.name
        ]

        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers:]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        if self.verbose:
            print("*** TRAINABLE VARS *** ")
            for var in self._trainable_weights:
                print(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        #inputs = [inputs.input_ids, inputs.input_mask, inputs.segment_ids]
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(input_ids=input_ids,
                           input_mask=input_mask,
                           segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs,
                           signature="tokens",
                           as_dict=True)["pooled_output"]
        """
        output = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)
        
        if self.pooling == "cls":
            pooled = output["pooled_output"]
        else:
            result = output["sequence_output"]
            
            input_mask = tf.cast(input_mask, tf.float32)
            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            
            if self.pooling == "mean":
              pooled = masked_reduce_mean(result, input_mask)
            else:
              pooled = mul_mask(result, input_mask)

        return pooled
        """

        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'bert_path': self.bert_path,
            'verbose': self.verbose,
            'n_fine_tune_layers': self.n_fine_tune_layers,
        })
        return config


# can take more fully model from https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb

In [0]:
# Build model
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    # Instantiate the custom Bert Layer defined above
    bert_output = BertLayer(BERT_MODEL_HUB, verbose=True, n_fine_tune_layers=5)(bert_inputs)

    # Build the rest of the classifier 
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(len(TARGET_COLS), activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

TODO: clear memory and increase batchsize

In [0]:
RocAucES = RocAucEarlyStopping(validation_data=([valid_input_ids, valid_input_masks, valid_segment_ids], valid_labels), 
                                   patience=1, digits=4)
RocAucCheckpointer = ModelRocAucCheckpoint(filepath=output_folder+'/BertModel_multilabel.h5',
                                           validation_data=([valid_input_ids, valid_input_masks, valid_segment_ids], valid_labels),
                                            save_best_only=True,
                                            save_weights_only=False,
                                            digits=3,
                                            save_freq=1,
                                            verbose=1)

In [0]:
gc.collect()

model = build_model(max_seq_length=MAX_WORDS_IN_COMMENT)

# Instantiate variables
initialize_vars(sess)

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([valid_input_ids, valid_input_masks, valid_segment_ids], valid_labels),
    epochs=10,
    batch_size=256,
    callbacks=[RocAucES, RocAucCheckpointer]
)

*** TRAINABLE VARS *** 
<tf.Variable 'bert_layer_module/bert/encoder/layer_9/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_module/bert/encoder/layer_9/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_module/bert/encoder/layer_9/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert_layer_module/bert/pooler/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_module/bert/pooler/dense/kernel:0' shape=(768, 768) dtype=float32>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 220)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 220)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 220)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

KeyboardInterrupt: ignored

In [0]:
# Clear and load model
del model
gc.collect()
model = build_model(MAX_WORDS_IN_COMMENT)
initialize_vars(sess)
model.load_weights(output_folder+'/BertModel_multilabel.h5')

*** TRAINABLE VARS *** 
<tf.Variable 'bert_layer_2_module/bert/encoder/layer_9/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_2_module/bert/encoder/layer_9/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_2_module/bert/encoder/layer_9/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert_layer_2_module/bert/pooler/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert_layer_2_module/bert/pooler/dense/kernel:0' shape=(768, 768) dtype=float32>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 220)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 220)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 220)]        0                                            
__________________________________________________________________________________________________
bert_layer_2 (BertLayer)        (None, 768)          108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [0]:
%%time

preprocessed_test_df = pd.DataFrame(preprocessed_test,
                                    columns=['preprocessed_text'])
test_input_ids, test_input_masks, test_segment_ids = text2input_arrays(
    text=preprocessed_test_df,
    tokenizer=bert_tokenizer,
    max_seq_length=MAX_WORDS_IN_COMMENT,
    labels_available=False)


CPU times: user 2min 13s, sys: 0 ns, total: 2min 13s
Wall time: 2min 12s


In [0]:
%%time

pred  = model.predict([test_input_ids, test_input_masks, test_segment_ids])
pred.shape

CPU times: user 13min 26s, sys: 9min 42s, total: 23min 8s
Wall time: 20min 49s


# Cheaty evaluating on test labels

In [0]:
submission = pd.read_csv(submission_path)
submission[TARGET_COLS] = (pred)

In [0]:
submission.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998786,4.020746e-01,0.994310,8.994406e-02,0.933397,0.148623
1,0000247867823ef7,0.000663,4.231930e-06,0.000808,3.576279e-07,0.000124,0.000005
2,00013b17ad220c46,0.000101,6.556511e-07,0.000130,4.768372e-07,0.000013,0.000003
3,00017563c3f7919a,0.000077,2.682209e-07,0.000032,3.427267e-06,0.000008,0.000002
4,00017695ad8997eb,0.000178,1.788139e-06,0.000064,7.629395e-06,0.000013,0.000004
5,0001ea8717f6de06,0.000252,2.384186e-07,0.000026,6.854534e-07,0.000026,0.000005
6,00024115d4cbde0f,0.000460,4.172325e-07,0.000081,3.576279e-07,0.000048,0.000002
7,000247e83dcc1211,0.837042,6.785691e-03,0.191099,1.680136e-03,0.193172,0.002311
8,00025358d4737918,0.292849,1.601279e-04,0.012059,1.319349e-04,0.101985,0.001267
9,00026d1092fe71cc,0.000117,4.470348e-07,0.000121,2.384186e-07,0.000016,0.000002


In [0]:
# labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)
test_labels_df = pd.read_csv(test_labels_filepath)

In [0]:
test_labels_df = test_labels_df[(test_labels_df["toxic"] != -1) &
                                (test_labels_df["severe_toxic"] != -1) &
                                (test_labels_df["obscene"] != -1) &
                                (test_labels_df["threat"] != -1) &
                                (test_labels_df["insult"] != -1) &
                                (test_labels_df["identity_hate"] != -1)]
test_labels_df.shape                               

(63978, 7)

In [0]:
submission_to_evaluate = submission[submission['id'].isin(test_labels_df['id'].values)]
submission_to_evaluate.shape

(63978, 7)

In [0]:
scores = []
for class_name in TARGET_COLS:
    train_target = test_labels_df[class_name]
    train_predicted = submission_to_evaluate[class_name]

    cv_score = np.mean(roc_auc_score(train_target.values, train_predicted.values))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9570823827972889
CV score for class severe_toxic is 0.9867829784722254
CV score for class obscene is 0.9729364125272775
CV score for class threat is 0.9832505960495843
CV score for class insult is 0.9674179960542009
CV score for class identity_hate is 0.9735440974739437
Total CV score is 0.97350241056242


In [0]:
submission.to_csv(os.path.join(output_folder,"submission_fine_tuned_bert.csv"), index = False)

Жирный шрифт означает изменения в процессе валидации и оценивания и при изменении действует начиная со строчки указания и ниже

* glove twitter 200:  0.98052

**С поднятием колва epochs и global_epochs + RocAucEarlyStopping** скор упал с 0.98052 до 0.97573

* GLOBAL_EPOCHS = 2, EPOCHS = 5, glove.840B.300d.txt   0.97994  
* BPEmb(lang="en", dim=25, vs=20000)   0.97375  
* BPEmb(lang="en", dim=300, vs=20000) without preprocessing   0.97699  
* BPEmb(lang="en", dim=300, vs=20000)   0.97865   
* fasttext crawl-300d-2M-subword 0.95732  
* BPEmb(lang="en", dim=300, vs=20000) **Patience 2->1 SEEDS 1->1**  0.98041   
* flair ELMoEmbeddings('small') embedding len 768  0.98005  
* flair ELMoEmbeddings('medium') embedding len 1536  0.98038  
* flair RoBERTaEmbeddings('roberta-base') embedding len 768  0.97413  
* flair stack of ELMoEmbeddings('small') + RoBERTaEmbeddings('roberta-base') embedding len 1536  0.98069      
* 3 tuned layers in BERT modeldense = tf.keras.layers.Dense(256, activation='relu')(bert_output)pred = tf.keras.layers.Dense(len(TARGET_COLS), activation='sigmoid')(dense)model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])epochs=1,batch_size=32   0.97658
* 10 tuned layers in BERT modeldense = tf.keras.layers.Dense(256, activation='relu')(bert_output)pred = tf.keras.layers.Dense(len(TARGET_COLS), activation='sigmoid')(dense)model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])epochs=1,batch_size=128, 5 epochs+ES(patience=1)  0.97377
* 5 tuned layers in BERT modeldense = tf.keras.layers.Dense(256, activation='relu')(bert_output)pred = tf.keras.layers.Dense(len(TARGET_COLS), activation='sigmoid')(dense)model.compile(loss='binary_crossentropy', optimizer='adam')epochs=10,batch_size=256, ES(patience=1) +save best  0.97346